# Neural Network N01
In this example, we will walk through a multi-input single output neural network model. We will train this model to classify images that contains cats and the images that don not.

The following contents will be covered:

1. Image data processing.
2. Construct a multi-input non-linear model.
3. Binary cross entropy function.


## Load Data
You may need to install h5py library in the terminal
```console
pip install h5py
```
Load training and test images and corresponding classes. 

In [ ]:
import h5py
import numpy as np

# Load training data
train_dataset = h5py.File("datasets/train_catvnoncat.h5", "r")
train_set_x = np.array(train_dataset["train_set_x"][:])
train_set_y = np.array(train_dataset["train_set_y"][:])
# Load test data
test_dataset = h5py.File("datasets/test_catvnoncat.h5", "r")
test_set_x = np.array(test_dataset["test_set_x"][:])
test_set_y = np.array(test_dataset["test_set_y"][:])
# Load text class list
classes = np.array(test_dataset["list_classes"][:])
# reshape labels
train_set_y = train_set_y.reshape((-1, 1))
test_set_y = test_set_y.reshape((-1, 1))

# Take a peek on data
print(train_set_x.shape, train_set_y.shape, test_set_x.shape, test_set_y.shape)
print(train_set_x[0])
print(train_set_y)


In [ ]:
# Visualize data
import matplotlib.pyplot as plt
plt.imshow(train_set_x[2])

## Preprocess the Data
1. Reshape/Flatten images
2. Rescale images 
3. (Already done.) Reshape targets/labels/classes/categories 

In [ ]:
# Process training data
X_train_flatten = train_set_x.reshape(train_set_x.shape[0], -1)
X_train = X_train_flatten / 255.  # rescale pixel value to 0. ~ 1.
y_train = train_set_y
# Process test data
X_test = test_set_x.reshape(test_set_x.shape[0], -1) / 255.
y_test = test_set_y

# Verify data preprocessing
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
print(X_train[0])


## Train a Model with MES Loss
1. Compute forward pass: input images features (pixels' values) to model, output probabilities of the being cat pictures.
2. Compute loss using mean square error function.
3. Compute gradients of loss w.r.t. weights and bias.
4. Compute backward pass: update weights and bias based on gradients. 

In [ ]:
def sigmoid(x):
    """ Sigmoid function
    Args:
        x: independent variable, could be an arrary of any shape or a scalar.
    Returns:
        y: dependent variable, could be an arrary of any shape or a scalar.
    """
    y = 1 / (1 + np.exp(-x))
    return y

# def d_sigmoid(x):
#     """ Derivative of sigmoid function
#     Args:
#         x: independent variable, could be an arrary of any shape or a scalar.
#     Returns:
#         dydx: dependent variable, could be an arrary of any shape or a scalar.
#     """
#     dydx = sigmoid(x) * (1 - sigmoid(x))
#     return dydx

def forward(input, weight, bias):
    """ Model function
    Args:
        input: feature matrix (independent variables), 2d-array with shape (# samples, # features)
        weight: row vector of weights, 2d-array with shape (1, # features)
        bias: scalar
    Returns:
        output: column vector of predictions (sigmoid activated outcomes), 2d-array with shape (# samples, 1)
        z: column vector of intermediate outputs, 2d-array with shape (# samples, 1)
    """
    z = np.dot(input, weight.T) + bias
    output = sigmoid(z)
    return output, z

def mse_loss(prediction, target):
    """ Mean Square Error function
    Args:
        prediction: column vector of predictions, 2d-array with shape (# samples, 1)
        target: column vector of ground truths, 2d-array with shape (# samples, 1)
    Returns:
        loss_value: scalar
    """
    loss_value = np.mean(1 / 2 * (prediction - target) ** 2)
    return loss_value

def grad(prediction, target, z, input):
    """ Gradient function with sigmoid activation
    Args:
        prediction: column vector of predictions, 2d-array with shape (# samples, 1)
        target: column vector of ground truths, 2d-array with shape (# samples, 1)
        z: column vector of intermediate outputs, 2d-array with shape (# samples, 1)
        feature: feature matrix, 2d-array with shape (# samples, # features)
    Returns:
        dL_dw: row vector of MSE loss partial derivatives w.r.t. weights, 2d-array with shape (1, # features)
        dL_db: scalar of MSE loss partial derivatives w.r.t. bias
    """
    dl_dyhat = prediction - target
    dl_dz = dl_dyhat * sigmoid(z) * (1 - sigmoid(z))
    dL_dw = 1 / target.shape[0] * np.dot(dl_dz.T, input)
    dL_db = dl_dz.mean()
    
    return dL_dw, dL_db

w = np.zeros((1, X_train.shape[-1])) + np.random.normal(0, 1e-4, (1, X_train.shape[-1]))
b = np.random.normal(0, 1e-4)
# Optimization
num_iters = 8000
learning_rate = 0.02
losses = []
for i in range(num_iters):
    pred_train, z = forward(X_train, w, b)
    dw, db = grad(pred_train, y_train, z, X_train)
    loss = mse_loss(pred_train, y_train)
    w = w - learning_rate * dw
    b = b - learning_rate * db
    print(f"loss @ {i+1} iteration: {loss}")
    # print(f"w = {w}, b = {b}")
    losses.append(loss)
# print(f"updated w={w}, b={b}")

# Observe loss values
plt.plot(losses)

## Test Model with MSE Loss

In [ ]:
cat_likelihood, _ = forward(X_test, w, b)
print(f"Model's mse on test data: {mse_loss(cat_likelihood, y_test)}")
yhat_test = np.zeros_like(cat_likelihood)
yhat_test[cat_likelihood>0.5] = 1
prediction_correctness = yhat_test==y_test
num_correct_preds = prediction_correctness.sum()
accuracy = num_correct_preds / y_test.shape[0]
print(accuracy)


## Train a Model with Cross Entropy Loss
1. Compute forward pass: input images features (pixels' values) to model, output probabilities of the being cat pictures.
2. Compute loss using binary cross entropy function (tailored to binary classification).
3. Compute gradients of loss w.r.t. weights and bias.
4. Compute backward pass: update weights and bias based on gradients. 

In [ ]:
def sigmoid(x):
    """ Sigmoid function
    Args:
        x: independent variable, could be an arrary of any shape or a scalar.
    Returns:
        y: dependent variable, could be an arrary of any shape or a scalar.
    """
    y = 1 / (1 + np.exp(-x))
    return y

# def d_sigmoid(x):
#     """ Derivative of sigmoid function
#     Args:
#         x: independent variable, could be an arrary of any shape or a scalar.
#     Returns:
#         dydx: dependent variable, could be an arrary of any shape or a scalar.
#     """
#     dydx = sigmoid(x) * (1 - sigmoid(x))
#     return dydx

def forward(input, weight, bias):
    """ Model function
    Args:
        input: feature matrix (independent variables), 2d-array with shape (# samples, # features)
        weight: row vector of weights, 2d-array with shape (1, # features)
        bias: scalar
    Returns:
        output: column vector of predictions (sigmoid activated outcomes), 2d-array with shape (# samples, 1)
        Z: column vector of intermediate outputs, 2d-array with shape (# samples, 1)
    """
    Z = np.dot(input, weight.T) + bias
    output = sigmoid(Z)
    return output, Z

def cross_entropy_loss(prediction, target):
    """ Cross Entropy function
    Args:
        prediction: column vector of predictions, 2d-array with shape (# samples, 1)
        target: column vector of ground truths, 2d-array with shape (# samples, 1)
    Returns:
        loss_value: scalar
    """
    loss_value = -np.mean(target * np.log(prediction) + (1 - target) * np.log(1 - prediction))
    return loss_value

def grad(prediction, target, z, input):
    """ Gradient function with sigmoid activation
    Args:
        prediction: column vector of predictions, 2d-array with shape (# samples, 1)
        target: column vector of ground truths, 2d-array with shape (# samples, 1)
        z: column vector of intermediate outputs, 2d-array with shape (# samples, 1)
        feature: feature matrix, 2d-array with shape (# samples, # features)
    Returns:
        dL_dw: row vector of MSE loss partial derivatives w.r.t. weights, 2d-array with shape (1, # features)
        dL_db: scalar of MSE loss partial derivatives w.r.t. bias
    """
    # dl_dyhat = -target / prediction + (1 - target) / (1 - prediction)
    # dl_dz = dl_dyhat * d_sigmoid(z)
    dl_dz = prediction - target
    dL_dw = 1 / target.shape[0] * np.dot(dl_dz.T, input)
    dL_db = dl_dz.mean()
    
    return dL_dw, dL_db

w = np.zeros((1, X_train.shape[-1])) + np.random.normal(0, 1e-4, (1, X_train.shape[-1]))
b = np.random.normal(0, 1e-4)
# Optimization
num_iters = 8000
learning_rate = 0.005
losses = []
for i in range(num_iters):
    pred_train, z = forward(X_train, w, b)
    dw, db = grad(pred_train, y_train, z, X_train)
    loss = cross_entropy_loss(pred_train, y_train)
    w = w - learning_rate * dw
    b = b - learning_rate * db
    print(f"loss @ {i+1} iteration: {loss}")
    # print(f"w = {w}, b = {b}")
    losses.append(loss)
# print(f"updated w={w}, b={b}")

# Observe loss values
plt.plot(losses)


## Test Model with Cross Entropy Loss

In [ ]:
cat_likelihood, _ = forward(X_test, w, b)
print(f"Model's ce on test data: {cross_entropy_loss(cat_likelihood, y_test)}")
yhat_test = np.zeros_like(cat_likelihood)
yhat_test[cat_likelihood>0.5] = 1
prediction_correctness = yhat_test==y_test
num_correct_preds = prediction_correctness.sum()
accuracy = num_correct_preds / y_test.shape[0]
print(accuracy)

## Test Model with New Images

In [ ]:
import cv2 as cv

image_raw = cv.imread('american_shorthair.jpg')  # read the raw image from a file
image_rgb = cv.cvtColor(image_raw, cv.COLOR_BGR2RGB)  # Convert BGR to RGB
image_resize = cv.resize(image_rgb, (64, 64))  # Resize image to shape (200, 200, 3)
image_flatten = image_resize.reshape(1, -1)  # Flatten image array to a row vector with shape (1, 200*200*3)
image_rescale = image_flatten / 255.  # rescale pixel value from 0~255 to 0.~1.
cat_likelihood, _ = forward(image_rescale, w, b)  # predict new image with your model

is_cat = cat_likelihood > 0.5
if is_cat.squeeze():
    print("It's a cat!") 
else:
    print("It's not a cat!")

plt.imshow(image_rgb) 
